# Core Walkthrough: Categories, Functors, Naturality, Monads

This notebook demonstrates non-agent features:
- Build small categories (terminal, discrete, simplex Δ³)
- Define and check functors
- Define and check natural transformations
- Use FP monads (`Id`, `Maybe`) and `Kleisli`
- Render Mermaid diagrams (categories, functors, naturality, plan)



In [55]:
from LambdaCat.core.standard import terminal_category, discrete, simplex, walking_isomorphism
from LambdaCat.core.laws import run_suite
from LambdaCat.core.laws_category import CATEGORY_SUITE

Term = terminal_category()
Disc = discrete(["A","B"]) 
Delta3 = simplex(3)
Iso = walking_isomorphism()

assert run_suite(Term, CATEGORY_SUITE).ok
assert run_suite(Disc, CATEGORY_SUITE).ok
assert run_suite(Delta3, CATEGORY_SUITE).ok
assert run_suite(Iso, CATEGORY_SUITE).ok

Term, Disc, Delta3, Iso


(Cat(objects=(Obj(name='*', data=None),), arrows=(ArrowGen(name='id:*', source='*', target='*'),), composition=mappingproxy({('id:*', 'id:*'): 'id:*'}), identities=mappingproxy({'*': 'id:*'})),
 Cat(objects=(Obj(name='A', data=None), Obj(name='B', data=None)), arrows=(ArrowGen(name='id:A', source='A', target='A'), ArrowGen(name='id:B', source='B', target='B')), composition=mappingproxy({('id:A', 'id:A'): 'id:A', ('id:B', 'id:B'): 'id:B'}), identities=mappingproxy({'A': 'id:A', 'B': 'id:B'})),
 Cat(objects=(Obj(name='0', data=None), Obj(name='1', data=None), Obj(name='2', data=None), Obj(name='3', data=None)), arrows=(ArrowGen(name='id:0', source='0', target='0'), ArrowGen(name='0->1', source='0', target='1'), ArrowGen(name='0->2', source='0', target='2'), ArrowGen(name='0->3', source='0', target='3'), ArrowGen(name='id:1', source='1', target='1'), ArrowGen(name='1->2', source='1', target='2'), ArrowGen(name='1->3', source='1', target='3'), ArrowGen(name='id:2', source='2', target='2'),

In [56]:
from LambdaCat.core.functor import FunctorBuilder
from LambdaCat.core.laws_functor import FUNCTOR_SUITE

F = (FunctorBuilder('F', source=Delta3, target=Iso)
     .on_objects({"0":"A","1":"A","2":"B","3":"B"})
     .on_morphisms({"0->1":"id:A","1->2":"f","2->3":"id:B","0->3":"f"})
     .build())

report = run_suite(F, FUNCTOR_SUITE)
print(report.to_text())
F.name, len(F.object_map), len(F.morphism_map)


Suite functor-core: OK
  - functor-identities: OK
  - functor-composition: OK


('F', 4, 10)

In [57]:
from LambdaCat.core.natural import Natural
from LambdaCat.core import NATURAL_SUITE

eta = Natural(source=F, target=F, components={"0":"id:A","1":"id:A","2":"id:B","3":"id:B"})
report = run_suite(eta, NATURAL_SUITE)
print(report.to_text())
list(eta.components.items())


Suite natural-core: OK
  - naturality-components-typed: OK
  - naturality-squares: OK


[('0', 'id:A'), ('1', 'id:A'), ('2', 'id:B'), ('3', 'id:B')]

In [58]:
from LambdaCat.core.fp.kleisli import Kleisli
from LambdaCat.core.fp.instances.maybe import Maybe
from LambdaCat.core.fp.instances.identity import Id

parse_int = Kleisli(lambda s: Maybe(int(s)) if str(s).isdigit() else Maybe(None))
recip = Kleisli(lambda n: Maybe(None) if n == 0 else Maybe(1.0 / n))
pipeline = parse_int.then(recip)

print(pipeline.run("12"))   # Maybe(1/12)
print(pipeline.run("oops"))  # Maybe(None)

# Identity monad
inc = Kleisli(lambda n: Id(n + 1))
dbl = Kleisli(lambda n: Id(n * 2))
print(inc.then(dbl).run(3))  # Id(8)



Maybe(value=0.08333333333333333)
Maybe(value=None)
Id(value=8)


In [59]:
from LambdaCat.extras.viz_mermaid import render_all, TwoCellView, category_mermaid, functor_mermaid, naturality_mermaid

md = render_all({
    'Delta3': Delta3,
    'Iso': Iso,
    'F': F,
    'eta': eta,
}, out_dir=None)

print(md['Delta3__category.md'][:200])
print(md['F__functor.md'][:200])



# Category

```mermaid
graph LR
  0 -- "0->1" --> 1
  0 -- "0->2" --> 2
  0 -- "0->3" --> 3
  1 -- "1->2" --> 2
  1 -- "1->3" --> 3
  2 -- "2->3" --> 3
```
# Functor

```mermaid
graph LR
subgraph Source
  S_0 -- "0->1" --> S_1
  S_0 -- "0->2" --> S_2
  S_0 -- "0->3" --> S_3
  S_1 -- "1->2" --> S_2
  S_1 -- "1->3" --> S_3
  S_2 -- "2->3" --> S_3
  S_0 -- 


## Opposite category and dual functors
We can construct the opposite category `C^op` and examine how a functor `F: C -> D` induces structure on `C^op`.


In [60]:
from LambdaCat.core.ops_category import opposite_category

Delta3_op = opposite_category(Delta3)
print(len(Delta3.objects), len(Delta3.arrows))
print(len(Delta3_op.objects), len(Delta3_op.arrows))


4 10
4 10


## Functor algebra: composition and identity
Demonstrate `FunctorBuilder` preserving identities and composition; verify with the law suite and inspect violations when broken.


In [61]:
from LambdaCat.core.functor import CatFunctor
from LambdaCat.core.laws_functor import FUNCTOR_SUITE

# Builder enforces laws and raises on invalid data. For a violations demo,
# construct a CatFunctor directly with an incomplete/incorrect morphism map.
broken = CatFunctor(
    name="B",
    source=Delta3,
    target=Iso,
    object_map={"0":"A","1":"A","2":"B","3":"B"},
    morphism_map={"0->1":"id:A"},  # intentionally incomplete
)
rep = run_suite(broken, FUNCTOR_SUITE)
print(rep.to_text())


Suite functor-core: FAIL
  - functor-identities: FAIL
    • ERROR F(id_0) ≠ id_A | witness={'X': '0'}
    • ERROR F(id_1) ≠ id_A | witness={'X': '1'}
    • ERROR F(id_2) ≠ id_B | witness={'X': '2'}
    • ERROR F(id_3) ≠ id_B | witness={'X': '3'}
  - functor-composition: FAIL
    • ERROR compose error: 'id:0' | witness={'g': 'id:0', 'f': 'id:0'}
    • ERROR compose error: 'id:0' | witness={'g': '0->1', 'f': 'id:0'}
    • ERROR compose error: '0->2' | witness={'g': '0->2', 'f': 'id:0'}
    • ERROR compose error: '0->3' | witness={'g': '0->3', 'f': 'id:0'}
    • ERROR compose error: 'id:1' | witness={'g': 'id:1', 'f': '0->1'}
    • ERROR compose error: '1->2' | witness={'g': '1->2', 'f': '0->1'}
    • ERROR compose error: '1->3' | witness={'g': '1->3', 'f': '0->1'}
    • ERROR compose error: 'id:2' | witness={'g': 'id:2', 'f': '0->2'}
    • ERROR compose error: '2->3' | witness={'g': '2->3', 'f': '0->2'}
    • ERROR compose error: 'id:3' | witness={'g': 'id:3', 'f': '0->3'}
    • ERROR co

In [62]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']


In [63]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']


In [64]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']


In [65]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']


In [66]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']


## Monads and Kleisli composition
We explore `Maybe` for partiality and `Id` for pure computation; show associativity via chaining.


In [67]:
from LambdaCat.core.natural import Natural
from LambdaCat.core import NATURAL_SUITE
bad = Natural(source=eta.source, target=eta.target, components={**eta.components, "1": "f"})
print(run_suite(bad, NATURAL_SUITE).to_text())


Suite natural-core: FAIL
  - naturality-components-typed: FAIL
    • ERROR η_1 has wrong type: expected A->A, got A->B | witness={'X': '1'}
  - naturality-squares: FAIL
    • ERROR Naturality failed on 0->1 | witness={'f': '0->1'}
    • ERROR Composition missing: 'composition not defined for (id:A,f)' | witness={'f': 'id:1'}
    • ERROR Composition missing: 'composition not defined for (f,f)' | witness={'f': '1->2'}
    • ERROR Composition missing: 'composition not defined for (f,f)' | witness={'f': '1->3'}


In [68]:
from LambdaCat.core.fp.kleisli import kleisli_identity
I = kleisli_identity(Id.pure)
print(I.run(5))  # Id(5)

# Associativity check (empirical)
from random import randint
f = Kleisli(lambda n: Id(n + 1))
g = Kleisli(lambda n: Id(n * 2))
h = Kleisli(lambda n: Id(n - 3))

for _ in range(3):
    x = randint(0, 10)
    left = f.then(g).then(h).run(x)
    right = f.then(g.then(h)).run(x)
    assert left == right
print("Kleisli associativity holds on samples.")


Id(value=5)
Kleisli associativity holds on samples.


## Serialization and diagrams
Convert a presentation to a plain dict and render diagrams for papers or PRs.


In [69]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']


In [70]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


{'objects': [{'name': 'A', 'data': None}, {'name': 'B', 'data': None}], 'arrows': [{'name': 'f', 'source': 'A', 'target': 'B'}, {'name': 'id:A', 'source': 'A', 'target': 'A'}, {'name': 'id:B', 'source': 'B', 'target': 'B'}], 'relations': []}
['Delta3__category.md', 'Iso__category.md']
